In [1]:
from tqdm import tqdm
import pdb
import limix
import pickle as pkl
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from numpy import asarray
import xarray as xr
from copy import deepcopy
import os
from os.path import join
import simplejson as json
import hashlib
import pandas as pd
import pdb
from glob import glob
import h5py
import re
from plot import plot_poisson_ordered, plot_distplots, plot_distplot
from util import get_amelie_selection
import h5py
import dask
import dask.dataframe
import dask.array
from urllib.request import urlopen
try:
    import ipfsapi
    has_ipfs = True
except ModuleNotFoundError:
    has_ipfs = False


if has_ipfs:
    try:
        ipfs = ipfsapi.connect('127.0.0.1', 5001)
    except Exception:
        has_ipfs = False

only_hash = True

/nfs/software/stegle/users/horta/conda-envs/rats/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Load data (Amelie's and Hannah's)

In [2]:
if os.path.exists("data/traits-kinship-hannah-amelie.pkl"):
    data = pkl.load(open("data/traits-kinship-hannah-amelie.pkl", "rb"))
else:
    url = "http://ipfs.io/ipfs/QmXtFuoA3JTkhhpUpqh4dyu3FJLUG7DWAmXwUk1N2TMbAh"
    data = pkl.load(urlopen(url))
    
dst_folder = "3"
if not os.path.exists(dst_folder):
    os.mkdir(dst_folder)
if not os.path.exists(join(dst_folder, 'phenotype')):
    os.mkdir(join(dst_folder, 'phenotype'))
if not os.path.exists(join(dst_folder, 'kinship')):
    os.mkdir(join(dst_folder, 'kinship'))

In [3]:
def isbernoulli(x):
    x = np.asarray(x, float)
    u = np.unique(x)
    if len(u) == 2:
        return True
    return False

def get_bernoulli(x):
    x = np.asarray(x, float)
    u = np.unique(x)
    i0 = x == u[0]
    i1 = x == u[1]
    x[i0] = 0.0
    x[i1] = 1.0
    return x

def isdiscrete(x):
    x = np.asarray(x, float)
    ok = np.isfinite(x)
    return all(x[ok] == np.asarray(x[ok], int))

def get_poisson(x):
    x = np.asarray(x, float)
    mi = min(x)    
    if mi < 0:
        x += -mi
    return x

def isnumber(x):
    try:
        np.asarray(x, float)
    except ValueError:
        return False
    return True

In [4]:
def analysis_QTL(G, pheno, kinship, pheno_name, lik, pheno_norm):
    common_samples = set(kinship.index.values).intersection(set(pheno.index.values))
    common_samples = np.asarray(list(common_samples))
    
    pheno = pheno.reindex(common_samples).copy()
    kinship = kinship.reindex(index=common_samples).copy()
    kinship = kinship.reindex(columns=common_samples).copy()
    
    assert(all(kinship.index.values == kinship.columns.values))
    assert(all(kinship.index.values == pheno.index.values))

    ok = np.isfinite(pheno.loc[:, pheno_name])
    common_samples = common_samples[ok]
    
    pheno = pheno.reindex(common_samples).copy()
    kinship = kinship.reindex(index=common_samples).copy()
    kinship = kinship.reindex(columns=common_samples).copy()
    
    assert(all(kinship.index.values == kinship.columns.values))
    assert(all(kinship.index.values == pheno.index.values))
    
    if not all(np.isfinite(pheno[pheno_name])):
        raise ValueError("not all finite: {}".format(pheno_name))
    
    y = pheno_norm(pheno.loc[:, pheno_name])
    y = pd.Series(y, index=pheno.loc[:, pheno_name].index)
    
#     phenotype_series = pd.Series(y, pheno.loc[:, pheno_name].index.values)
    obj = pkl.dumps(y)
    m = hashlib.sha1()
    m.update(obj)
    phenotype_hex = m.hexdigest()
    with open(join(dst_folder, "phenotype", phenotype_hex + ".series.pkl"), "wb") as f:
        f.write(obj)
    
    obj = pkl.dumps(kinship)
    m = hashlib.sha1()
    m.update(obj)
    kinship_hex = m.hexdigest()
    with open(join(dst_folder, "kinship", kinship_hex + ".dataframe.pkl"), "wb") as f:
        f.write(obj)

    try:
        call = "limix.qtl.scan(G, y, lik, kinship, verbose=True)"
        model = limix.qtl.scan(G, y, lik, kinship, verbose=True)
    except ValueError as e:
        print(e)
        print("Pheno name: {}".format(pheno_name))
        return None

    return dict(pv=list(asarray(model.variant_pvalues)), lik=lik, call=call, phenotype=phenotype_hex, kinship=kinship_hex,
                null_covariate_effsizes=list(asarray(model.null_covariate_effsizes)),
                variant_effsizes=list(asarray(model.variant_effsizes)), variant_effsizes_se=list(asarray(model.variant_effsizes_se)))

In [5]:
f = h5py.File("arrayexpress/HS.hdf5", "r")
samples = np.asarray(f["/imputed_genotypes/row_header/rat"].value)
samples = [i.decode() for i in samples]
f.close()

In [6]:
data0 = deepcopy(data)
remove = []
for name in data0['measures'].columns.values:
    if isnumber(data0['measures'].loc[:, name]) and isdiscrete(data0['measures'].loc[:, name]):
        data0['measures'].loc[:, name] = get_poisson(data0['measures'].loc[:, name])
    else:
        remove.append(name)
for name in remove:
    del data0['measures'][name]

In [7]:
Gs = []
for i in tqdm(range(1, 22)):
    G = xr.open_dataarray("arrayexpress/HS.hdf5", "/imputed_genotypes/chr{}".format(i),
                          chunks=1000)
    G = G.rename({G.dims[0]: "snps", G.dims[1]: "samples"})
    Gs.append(G)

100%|██████████| 21/21 [00:01<00:00, 20.69it/s]


In [8]:
G = xr.concat(Gs, dim="snps").T
G['samples'] = samples
G['snps'] = range(G.shape[1])

In [9]:
# for name in data0['measures'].columns.values:
#     print("Processing: {}".format(name))
#     dst_file = join(dst_folder, "scan_measures_normal_" + name + "_mean_standardize.json")
#     if os.path.exists(dst_file) or os.path.exists(dst_file + ".failed"):
#         continue
#     r = analysis_QTL(G, data0['measures'], data0['kinship'], name, 'normal', limix.qc.mean_standardize)
#     if r is None:
#         open(dst_file + ".failed", "w").write("")
#     else:
#         json.dump(r, open(dst_file, "w"))

In [ ]:
for name in data0['measures'].columns.values:
    print("Processing: {}".format(name))
    dst_file = join(dst_folder, "scan_measures_normal_" + name + "_quantile_gaussianize.json")
    if os.path.exists(dst_file) or os.path.exists(dst_file + ".failed"):
        continue
    r = analysis_QTL(G, data0['measures'], data0['kinship'], name, 'normal', limix.qc.quantile_gaussianize)
    if r is None:
        open(dst_file + ".failed", "w").write("")
    else:
        json.dump(r, open(dst_file, "w"))

Processing: Boli_tot


============================= qtl analysis session starts ==============================

Normalising input... 

/nfs/software/stegle/users/horta/conda-envs/rats/lib/python3.6/site-packages/xarray/core/indexing.py:1168: PerformanceWarning: Slicing with an out-of-order index is generating 244 times more chunks
  return self.array[key]


done (5.73 seconds).


...

10it [00:00, 61.88it/s]

Scale: 1.19793
Delta: 0.5
Beta: [-0.]
Scale: 1.19755
Delta: 0.5
Beta: [-1.01495304]
Scale: 1.19755
Delta: 0.5
Beta: [-1.01495285]
Scale: 1.19755
Delta: 0.500001
Beta: [-1.01495246]
Scale: 1.19755
Delta: 0.500001
Beta: [-1.01495169]
Scale: 1.19755
Delta: 0.500002
Beta: [-1.01495014]
Scale: 1.19755
Delta: 0.500005
Beta: [-1.01494704]
Scale: 1.19754
Delta: 0.500009
Beta: [-1.01494083]
Scale: 1.19753
Delta: 0.500019
Beta: [-1.01492843]
Scale: 1.1975
Delta: 0.500038
Beta: [-1.01490363]
Scale: 1.19745
Delta: 0.500076
Beta: [-1.01485402]
Scale: 1.19735
Delta: 0.500153
Beta: [-1.01475482]
Scale: 1.19715
Delta: 0.500305
Beta: [-1.01455644]
Scale: 1.19674
Delta: 0.50061
Beta: [-1.01415978]
Scale: 1.19593
Delta: 0.501221
Beta: [-1.01336691]
Scale: 1.19431
Delta: 0.502441
Beta: [-1.01178292]
Scale: 1.1911
Delta: 0.504883
Beta: [-1.00862192]
Scale: 1.18475
Delta: 0.509765
Beta: [-1.00232786]
Scale: 1.17238
Delta: 0.519522
Beta: [-0.98985132]
Scale: 1.14886
Delta: 0.538985
Beta: [-0.96534333]
Scale:

37it [00:00, 170.83it/s]


Scale: 0.949207
Delta: 0.789542
Beta: [-0.45713023]
Scale: 0.949207
Delta: 0.789542
Beta: [-0.45713084]
Scale: 0.949207
Delta: 0.789541
Beta: [-0.45713084]
Scale: 0.949207
Delta: 0.789541
Beta: [-0.45713096]
Scale: 0.949207
Delta: 0.789542
Beta: [-0.45713089]
Scale: 0.949207
Delta: 0.789542
Beta: [-0.45713086]
Scale: 0.949207
Delta: 0.789542
Beta: [-0.45713087]



Scanning:   7%|▋         | 2/30 [02:52<40:20, 86.44s/it]

In [ ]:
f.close()